In [1]:
""" Purpose of this program is to load ticker/quote data into a local postgres database, eventually this may
be expanded to something like CosmosDB.
"""
import psycopg2
import pandas as pd
import yahoo_fin.stock_info as si
import time

In [2]:
conn = psycopg2.connect("dbname=StonksGoUp user=postgres host=localhost password=admin")
cur = conn.cursor()
data = si.get_data('msft')
SQL_stockdata = """ 
    CREATE TABLE stockdata (
        ticker varchar(5) NOT NULL,
        quotedate date NOT NULL,
        open numeric NOT NULL,
        high numeric NOT NULL,
        low numeric NOT NULL,
        close numeric NOT NULL,
        adjclose numeric NOT NULL,
        volume bigint,
        CONSTRAINT pk_stockdata PRIMARY KEY (ticker, quotedate)
);
"""
SQL_tickers = """
    CREATE TABLE tickers (
    ticker varchar(10) NOT NULL,
    tradedindex varchar(10) NOT NULL, 
    createddate timestamp with time zone NOT NULL DEFAULT now(),
    CONSTRAINT pk_tickers PRIMARY KEY (ticker)
    );
"""
# cur.execute(SQL_stockdata, conn)
# cur.execute(SQL_tickers, conn)
# conn.commit()




In [3]:
"""INSERT data into the tickers reference table"""
df_ticker = pd.DataFrame(columns=['Ticker'])
nasdaq = si.tickers_nasdaq()
sp500 = si.tickers_sp500()
dow = si.tickers_dow()
other = si.tickers_other()
indexes = [nasdaq, sp500, dow, other]

# for ticker in nasdaq:
#     df_ticker = df_ticker.append({'Ticker' : ticker} , ignore_index=True)
for ticker in sp500:
    df_ticker = df_ticker.append({'Ticker' : ticker} , ignore_index=True)
# for ticker in dow:
#     df_ticker = df_ticker.append({'Ticker' : ticker} , ignore_index=True)
# for ticker in other:
#     df_ticker = df_ticker.append({'Ticker' : ticker , 'TradedIndex' : 'other'} , ignore_index=True)

df_ticker.dropna(inplace=True)
print(df_ticker.head())
insert = [list(row) for row in df_ticker.itertuples(index=False)]

# SQL_Ticker_insert= """ INSERT INTO public.tickers(ticker) VALUES (%s) ON CONFLICT DO NOTHING"""
# cur.executemany(SQL_Ticker_insert, insert)
# conn.commit()

# print(f'{cur.rowcount} rows inserted.')

  Ticker
0      A
1    AAL
2    AAP
3   AAPL
4   ABBV


In [5]:
"""INSERT quote data into stockdata table"""

SQL_stockdata_insert = """INSERT INTO public.stockdata (quotedate, open, high, low, close, adjclose, volume, ticker)
                            VALUES(%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT DO NOTHING"""
for ticker in df_ticker['Ticker']:
    time.sleep(3)
    print(ticker)
    try: 
        data = si.get_data(ticker)
        insert_data = [list(row) for row in data.itertuples()]
        cur.executemany(SQL_stockdata_insert, insert_data)
    except: pass
    conn.commit()

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTLT
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HFC
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
I

In [ ]:
cur.close()
conn.close()